In [5]:
from sklearn.kernel_approximation import *

class CycleKernel(BaseEstimator, TransformerMixin):

    def __init__(self, sigma=-1, num_components=100):
        self.sigma, self.num_components = sigma, num_components

    def fit(self, X, y=None):
        self.red = []
        if self.sigma > 0:
            self.approx = RBFSampler(gamma=self.sigma, n_components=self.num_components, random_state=10).fit(X[0][0])
        card = []
        for i in range(len(X)):
            mat, count = [], 0
            for cyc in range(len(X[i])):
                    count += 1
                    if self.sigma == -1:
                        mat.append(  np.mean(X[i][cyc], axis=0)[np.newaxis,:]  )
                    else:
                        mat.append(  np.mean(self.approx.transform(X[i][cyc]), axis=0)[np.newaxis,:]  )
            card.append(count)
            self.red.append( np.concatenate(mat, axis=0) )
        return self

    def transform(self, X):
        mat = np.zeros([len(X), len(self.red)])
        if len(self.red) == len(X):
            for i in range(len(X)):
                for j in range(i, len(X)):
                    mat[i,j] = np.mean(np.matmul(self.red[i], self.red[j].T))
                    mat[j,i] = mat[i,j]
        else:
            red2 = []
            for i in range(len(X)):
                mat2 = []
                for cyc in range(len(X[i])):
                        if self.sigma == -1:
                            mat2.append(  np.mean(X[i][cyc], axis=0)[np.newaxis,:]  )
                        else:
                            mat2.append(  np.mean(self.approx.transform(X[i][cyc]), axis=0)[np.newaxis,:]  )
                red2.append( np.concatenate(mat2, axis=0) )
            for i in range(len(X)):
                for j in range(len(self.red)):
                    mat[i,j] = np.mean(np.matmul(red2[i], self.red[j].T))
        return mat